# Predicting from a pre-trained motion generating model

### Running the notebook:


In [1]:
from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras import backend as K
from tensorflow.compat.v1.keras.layers import Dense, Input
import tensorflow.compat.v1 as tf
import numpy as np
import mdn
import datetime

import csv
from IPython.display import HTML, display

## Load a small dataset to use when generating new motion

In [2]:
datafolder = '1004zipped_testSCALED_yseqtest_seqlen256.npz'

loaded = np.load(datafolder)
x = loaded['x']
#y = loaded['y']

x = x.astype('float32', copy=False)
#y = y.astype('float32', copy=False)

print('x.shape: ', x.shape)
#print('y.shape: ', y.shape)

x.shape:  (14, 256, 66)


## Load the pre-trained model

In [3]:
HIDDEN_UNITS1 = 1024 # number of hidden units - ideally Id like to try 1024 buy OOM error..
HIDDEN_UNITS2 = 512
HIDDEN_UNITS3 = 256
N_MIXES =  4 # number of mixture components
INPUT_DIMS = 66 # 22 joints * 3 
OUTPUT_DIMS = 66  # number of real-values predicted by each mixture component
SEQ_LEN = 256 # Number of frames in an example
lr = 1e-5
opt = keras.optimizers.Adam(learning_rate=lr)
freq = 30 # frame rate of data




decoder = keras.Sequential()
decoder.add(keras.layers.LSTM(HIDDEN_UNITS1, batch_input_shape=(1,SEQ_LEN,INPUT_DIMS), return_sequences=True, stateful=True))#,batch_input_shape=(1,SEQ_LEN,INPUT_DIMS),input_shape=(SEQ_LEN,INPUT_DIMS)))
decoder.add(keras.layers.LSTM(HIDDEN_UNITS2, batch_input_shape=(1,SEQ_LEN,INPUT_DIMS), return_sequences=True, stateful=True))
decoder.add(keras.layers.LSTM(HIDDEN_UNITS3, stateful=True))
decoder.add(mdn.MDN(OUTPUT_DIMS, N_MIXES))
decoder.add(keras.layers.Activation('linear', dtype='float32'))
decoder.compile(loss=mdn.get_mixture_loss_func(OUTPUT_DIMS,N_MIXES), optimizer=opt)
decoder.summary()

model_name = 'DD-SCALED-units-mixed-mixtures4-drop0.2-lr1e-05-seqlen256' #DD-SCALED-units-mixed-mixtures3-drop0.2-lr1e-05-seqlen256'
decoder.load_weights(model_name+'.h5') # load weights independently from file

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (1, 256, 1024)            4468736   
_________________________________________________________________
lstm_1 (LSTM)                (1, 256, 512)             3147776   
_________________________________________________________________
lstm_2 (LSTM)                (1, 256)                  787456    
_________________________________________________________________
mdn (MDN)                    (1, 532)                  136724    
_________________________________________________________________
activation (Activation)      (1, 532)                  0         
Total params: 8,540,692
Trainable params: 8,540,692
Non-trainable params: 0
_________________________________________________________________


# Predicting motion


In [4]:
def write_ex_to_tsv(ex, fn):
    num_frames = ex.shape[0]
    marker_names = ['MARKER_NAMES','Head','neck','lsho','lelb','lwri','lhan','rsho','relb','rwri','rhan','t10','root','lhip','lknee','lank','lfoot','ltoe','rhip','rknee','rank','rfoot','rtoe']
    
    
    with open(fn+'.tsv', 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(['NO_OF_FRAMES', num_frames])
        tsv_writer.writerow(['NO_OF_CAMERAS', 0])
        tsv_writer.writerow(['NO_OF_MARKERS', 22])
        tsv_writer.writerow(['FREQUENCY', freq])
        tsv_writer.writerow(['NO_OF_ANALOG', 0])
        tsv_writer.writerow(['ANALOG_FREQUENCY', 0])
        tsv_writer.writerow(['DESCRIPTION--', ''])
        tsv_writer.writerow(['TIME_STAMP--', ''])
        tsv_writer.writerow(['DATA_INCLUDED', '3D'])
        tsv_writer.writerow(marker_names)
        
        for frame in range(num_frames):
            tsv_writer.writerow(ex[frame,:])

            
# Predict marker positions based on **sequence** of prev frames and save the result as tsv.
def shift(arr, num, fill_value=np.nan):
    #print(fill_value.shape)
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    #print(result.shape)
    return result


def predict_sequence(model, pi=1e-7, sigma=1e-7, frames=256, primer_idx=0, select_mix=False, use_priming=False, mix=0):
    motion = []
    idx = primer_idx
    pred_on = x[idx,:,:] # starting pose 
    
    for i in range(frames):
        reshaped_pred_on = tf.reshape(pred_on,[1,SEQ_LEN,OUTPUT_DIMS])
        params = decoder.predict(reshaped_pred_on, steps=1)
        
        if select_mix:
            pred = mdn.sample_from_output_select_mix(params[0], OUTPUT_DIMS, N_MIXES, temp=pi, sigma_temp=sigma, mix=mix)
        else:
            pred = mdn.sample_from_output(params[0], OUTPUT_DIMS, N_MIXES, temp=pi, sigma_temp=sigma)
            
        motion.append(pred.reshape((OUTPUT_DIMS,)))
        
        if use_priming:
            #print(idx)
            if i%SEQ_LEN==0:
                idx += 1
            #print(str(idx) +' '+ str(i%SEQ_LEN))
            pred_on = shift(pred_on, -1, fill_value=x[idx,i%SEQ_LEN,:])
        else:
            pred_on = shift(pred_on, -1,  fill_value=pred)
    
    motion = np.array(motion)
    date_string = datetime.datetime.today().strftime('%Y%m%d')
    fn = date_string+ '-pi_temp-' +str(pi) + '-sig_temp-' + str(sigma) + "-mix-" + str(mix) + "-primer_idx-" + str(primer_idx)
    
    if use_priming:
        fn = fn+'PRIMING'
    if select_mix:
        fn = fn+'MIX'
    
    print('Generated motion ', motion.shape, 'with filename ', fn)
    write_ex_to_tsv(motion,fn)
    
    return fn

In [5]:
def display_animation(var_film):
    var_film = var_film+'.mp4'

    link_t = "<div align='middle'><video width='80%' controls><source src='{href}' type='video/mp4'></video></div>"


    # create HTML object, using the string template
    html = HTML(link_t.format(href=var_film))

    # display the HTML object to put the link on the page:
    display(html)

# Sampling from isolated mixture components

In [6]:
fnmix1 = predict_sequence(decoder,frames=128, select_mix=True, mix=1)
fnmix2 = predict_sequence(decoder,frames=128, select_mix=True, mix=2)
fnmix3 = predict_sequence(decoder,frames=128, select_mix=True, mix=3)
fnmix0 = predict_sequence(decoder,frames=128, select_mix=True, mix=0)

Generated motion  (128, 66) with filename  20200421-pi_temp-1e-07-sig_temp-1e-07-mix-1-primer_idx-0MIX
Generated motion  (128, 66) with filename  20200421-pi_temp-1e-07-sig_temp-1e-07-mix-2-primer_idx-0MIX
Generated motion  (128, 66) with filename  20200421-pi_temp-1e-07-sig_temp-1e-07-mix-3-primer_idx-0MIX
Generated motion  (128, 66) with filename  20200421-pi_temp-1e-07-sig_temp-1e-07-mix-0-primer_idx-0MIX


In [7]:
%get fnmix3 fnmix0 fnmix1 fnmix2
build_animation(fnmix1);
build_animation(fnmix2);
build_animation(fnmix3);
build_animation(fnmix0);


fn =

    '20200421-pi_temp-1e-07-sig_temp-1e-07-mix-1-primer_idx-0MIX.tsv'


fn =

    '20200421-pi_temp-1e-07-sig_temp-1e-07-mix-2-primer_idx-0MIX.tsv'


fn =

    '20200421-pi_temp-1e-07-sig_temp-1e-07-mix-3-primer_idx-0MIX.tsv'


fn =

    '20200421-pi_temp-1e-07-sig_temp-1e-07-mix-0-primer_idx-0MIX.tsv'



In [8]:
display_animation(fnmix1)
display_animation(fnmix2)
display_animation(fnmix3)
display_animation(fnmix0)

# Sampling with temperature adjustment
As pi temperature increases towards 1.0 we are increasing the probability of sampling from different mixture components at each time step.

As sigma temperature increses towards 1.0 we are increasing the probability of sampling further from the mean of each mixture component at each time step.

### Low pi, low sigma

In [22]:
low_pi_sig = predict_sequence(decoder,frames=256,pi=1e-9,sigma=1e-9, primer_idx=10)

Generated motion  (256, 66) with filename  20200421-pi_temp-1e-09-sig_temp-1e-09-mix-0-primer_idx-10


In [23]:
%get low_pi_sig
build_animation(low_pi_sig);


fn =

    '20200421-pi_temp-1e-09-sig_temp-1e-09-mix-0-primer_idx-10.tsv'



In [24]:
display_animation(low_pi_sig)

### High pi, high sigma

In [25]:
high_pi_sig = predict_sequence(decoder,frames=256,pi=1.0,sigma=1.0, primer_idx=10)

Generated motion  (256, 66) with filename  20200421-pi_temp-1.0-sig_temp-1.0-mix-0-primer_idx-10


In [26]:
%get high_pi_sig
build_animation(high_pi_sig);


fn =

    '20200421-pi_temp-1.0-sig_temp-1.0-mix-0-primer_idx-10.tsv'



In [27]:
display_animation(high_pi_sig)

### Low pi, high sigma

In [28]:
low_pi_high_sig = predict_sequence(decoder,frames=256,pi=1e-9,sigma=1.0, primer_idx=10)

Generated motion  (256, 66) with filename  20200421-pi_temp-1e-09-sig_temp-1.0-mix-0-primer_idx-10


In [29]:
%get low_pi_high_sig
build_animation(low_pi_high_sig);


fn =

    '20200421-pi_temp-1e-09-sig_temp-1.0-mix-0-primer_idx-10.tsv'



In [30]:
display_animation(low_pi_high_sig)

### High pi, low sigma

In [31]:
high_pi_low_sig = predict_sequence(decoder,frames=256,pi=1.0,sigma=1e-9, primer_idx=10)

Generated motion  (256, 66) with filename  20200421-pi_temp-1.0-sig_temp-1e-09-mix-0-primer_idx-10


In [32]:
%get high_pi_low_sig
build_animation(high_pi_low_sig);


fn =

    '20200421-pi_temp-1.0-sig_temp-1e-09-mix-0-primer_idx-10.tsv'



In [33]:
display_animation(high_pi_low_sig)

# Sampling with priming
priming the model on a motion example 

## Original recordings: Dance 1

In [34]:
write_ex_to_tsv(x[4,:,:],'dance1')
dance1 = 'dance1'

In [35]:
%get dance1
build_animation(dance1);


fn =

    'dance1.tsv'



In [36]:
display_animation('dance1')

## Original recordings: Dance 2

In [37]:
write_ex_to_tsv(x[10,:,:],'dance2')
dance2 = 'dance2'

In [38]:
%get dance2
build_animation(dance2);


fn =

    'dance2.tsv'



In [39]:
display_animation('dance2')

### Generated with priming
primer idx 0-6 = Example A, dance 1. Performed to an upbeat song with a strong pulse.


primer idx 7-13 = Example B, dance 2. Performed to a slow, melodic song without a strong pulse.

In [40]:
primeA = predict_sequence(decoder,frames=256, use_priming=True, primer_idx=4)

Generated motion  (256, 66) with filename  20200421-pi_temp-1e-07-sig_temp-1e-07-mix-0-primer_idx-4PRIMING


In [41]:
%get primeA
build_animation(primeA);


fn =

    '20200421-pi_temp-1e-07-sig_temp-1e-07-mix-0-primer_idx-4PRIMING.tsv'



In [46]:
display_animation(primeA)

In [43]:
primeB = predict_sequence(decoder,frames=256, use_priming=True, primer_idx=10)

Generated motion  (256, 66) with filename  20200421-pi_temp-1e-07-sig_temp-1e-07-mix-0-primer_idx-10PRIMING


In [44]:
%get primeB
build_animation(primeB);


fn =

    '20200421-pi_temp-1e-07-sig_temp-1e-07-mix-0-primer_idx-10PRIMING.tsv'



In [47]:
display_animation(primeB)